## 初始化

In [ ]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()
# 添加cache 
# 以下要求NotebookCacheManager.py 和 cache目录都在 当前测试文件的相同目录下（在notebook目录下）
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e2")

from pprint import pprint

# cache使用示例： 
# cache.save('存储文件名', 参数)，  a_loaded = cache.load('存储文件名')， 存在pkl文件下，点开为乱码
# cache.save_json, cache.load_json, 存在json文件下，点开可读
# cache.save_md, cache.load_md，存在md文件下，点开可读
# cache.save_json('outline1/test1') 可构建二级目录

## PART-1. 进入Outline_L1 分析


In [2]:
# 说明： 
# 对于长文本，直接提取多个层级的标题效果不佳且不稳定。因此，我们将分步提取标题，先提取一级标题，然后二级标题，最后三级标题。<br>
# 为进行测试，我们需要：<br> 
# 1. 项目容器： 测试项目1 <br>
# 2. tender_file_extraction 准备就绪 <br>
# 3. 模型输入： context, instruction, supplement, output_format, prompt_template, index_path_map，都是即时生成，不存储。<br>
# 4. 调用模型的函数：从task中简化。<br>
# 5. 章节的截取函数： TiptapUtils.extract_indexed_paragraphs， 在prepare_context中已调用<br>
# 6. 任务结果L1： 初始化为空 <br>
# 7. 任务结果L2： 初始化为空 <br>
# 8. 任务结果L3： 初始化为空 <br>
# </font>

#### <font color="red"> 1. 准备tender_file_extraction</font>

In [ ]:
# 招标文件提取内容 准备就绪
from apps.projects.models import Project, Task, TaskType
from pprint import pprint
project = Project.objects.get(project_name='test2')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
stage = task.stage
# --- 如果project.tender_file_extraction 为空，则需要从关联的docx_extraction_task中获取
# extraction_task = Task.objects.get(stage__project=project, type=TaskType.DOCX_EXTRACTION_TASK)
# print(extraction_task.docx_tiptap)
# print(type(extraction_task.docx_tiptap))
# pprint(extraction_task.docx_tiptap)
# project = Project.objects.get(project_name='测试项目1')
# project.tender_file_extraction = extraction_task.docx_tiptap
# project.save()
# project = Project.objects.get(project_name='测试项目1')
pprint(project.tender_file_extraction)
cache.save_json('tender_file_extraction', project.tender_file_extraction)

In [ ]:
# 存储位就绪
print(f"outline_L1存储：{project.outline_L1}")
print(f"outline_L2存储：{project.outline_L2}")
print(f"outline_L3存储：{project.outline_L3}")
print(f"outline_L1存储：{project.index_path_map_L1}")
print(f"outline_L2存储：{project.index_path_map_L2}")
print(f"outline_L3存储：{project.index_path_map_L3}")

#### <font color="red"> 2. 分析前数据准备 model_params_l1, task_l1, meta_l1</font>

In [4]:
tender_file_extraction = cache.load_json('tender_file_extraction')
from pprint import pprint

In [ ]:
# 借用outline_analysis_L1.py 的函数， 进行任务准备 

# 集成时，用以下代替，模型传参 会先存储到数据库。
from apps.projects.services.tasks_preparation.outline_analysis_L1 import OutlineAnalysisL1
# #获取LLM分析相关数据
data_cooker_l1 = OutlineAnalysisL1(tender_file_extraction)
model_params_l1, task_l1, meta_l1 = data_cooker_l1.output_params()
# # 模拟的prompt, 
simulated_prompt, formatted_message = data_cooker_l1.simulate_prompt()
cache.save_json('outline_analysis_L1/model_params_l1', model_params_l1)
cache.save_json('outline_analysis_L1/task_l1', task_l1)
cache.save_json('outline_analysis_L1/meta_l1', meta_l1)
cache.save_json('outline_analysis_L1/formatted_message', formatted_message)


#### <font color="red"> 3. 大模型分析</font>

In [ ]:
# 构建LLM分析器 并 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
analyzer = LLMService(model_params_l1)
outline_L1 = await analyzer.process(task_l1)
pprint(outline_L1)


In [8]:
cache.save_md('outline_analysis_L1/outline_L1', outline_L1)

#### <font color="red"> 4. 将Outline_L1 应用到tender_file_extraction中</font>



In [9]:
# 应用outline_L1分析结果，数据准备： outline_L1, index_path_map_l1, tender_file_extraction 
import json
outline_L1 = cache.load_md('outline_analysis_L1/outline_L1')
meta_l1 = cache.load_json('outline_analysis_L1/meta_l1')
tender_file_extraction = cache.load_json('tender_file_extraction')
index_path_map_L1 = meta_l1['index_path_map']
outline_L1_dict = json.loads(outline_L1)

In [ ]:
# 将outline_L1 应用到tender_file_extraction中, 使用TiptapUtils.update_titles_from_list
from apps.projects.tiptap.helpers import TiptapUtils
updated_tiptap_content = TiptapUtils.update_titles_from_list(tender_file_extraction, outline_L1_dict, index_path_map_L1)
# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(updated_tiptap_content)
pprint(headings)
print(TiptapUtils.print_headings(updated_tiptap_content))

In [11]:
tender_file_extraction_L1 = updated_tiptap_content
cache.save_json('tender_file_extraction_L1', tender_file_extraction_L1)



## PART-2. 开始执行Outline_L2 分析


In [ ]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()
# 添加cache 
# 以下要求NotebookCacheManager.py 和 cache目录都在 当前测试文件的相同目录下（在notebook目录下）
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e2")

from pprint import pprint

# cache使用示例： 
# cache.save('存储文件名', 参数)，  a_loaded = cache.load('存储文件名')， 存在pkl文件下，点开为乱码
# cache.save_json, cache.load_json, 存在json文件下，点开可读
# cache.save_md, cache.load_md，存在md文件下，点开可读
# cache.save_json('outline1/test1') 可构建二级目录

In [27]:
# 说明： 使用TiptapUtils.extract_chapters 提取章节， 并根据章节分块， 好处是,index被保留。<br> 
# - 带有L1标题的tiptap JSON 内容已经ready <br>
# - 我们需要解决：<br>
#     1）提取章节？-> 在原文 基础上 提取段落内容（str）， 根据章节分块， 好处是,index被保留。<br>
#     2）提供章节段落内容，重复类似Outline_L1的分析: <br>
#         a. 准备容器，更新的文件内容，存储任务结果字段？<br>
#         b. 准备大模型的输入，并调用模型？ (这个过程，需要考虑并发分析，提高效率)<br>
#         c. 将结果应用到tiptap JSON的 内容中？<br>
#         d. 存储结果到project.tender_file_extraction <br>


In [12]:
tender_file_extraction_L1 =cache.load_json('tender_file_extraction_L1')

In [ ]:
# 提取章节
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_L1)
print(headings)

#### <font color="red"> 1. 通过outline_analysis_L2 进行传参准备</font>
<font color="orange" size="3">
- 在outline_analysis_L2 中， 根据章节分块， index被保留。<br>
- 使用了TiptapUtils.extract_chapters 工具，返回 字典对象chapter_set{}， 包含chapters 和 index_path_map 两个键名<br>
- 在outline_analysis_L2 中， 使用chapter_set['chapters'] for循环 取得chapter['paragraphs'] 作为context<br>
- 在outline_analysis_L2 的输出，将每个章节分析所需的传参 单独打包， 并返回 列表对象param_set[]<br>
</font>

In [14]:
# 集成时，用以下代替，模型传参 会先存储到数据库。
from apps.projects.services.tasks_preparation.outline_analysis_L2 import OutlineAnalysisL2
# #获取LLM分析相关数据
data_cooker_l2 = OutlineAnalysisL2(tender_file_extraction_L1)
model_params_l2, tasks_l2, meta_l2 = data_cooker_l2.output_params()
# # 模拟的prompt, 
simulated_prompts_l2, formatted_messages_l2 = data_cooker_l2.simulate_prompt()
cache.save_json('outline_analysis_L2/model_params_l2', model_params_l2)
cache.save_json('outline_analysis_L2/tasks_l2', tasks_l2)
cache.save_json('outline_analysis_L2/meta_l2', meta_l2)
cache.save_json('outline_analysis_L2/formatted_message', formatted_messages_l2)

#### <font color="red"> 2. 大模型逐个章节进行分析</font>

In [ ]:
# 构建LLM分析器 并 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
analyzer = LLMService(model_params_l2)
outlines_L2 = await analyzer.process_with_limit(tasks_l2, limit=5)
pprint(outlines_L2)

# 存储OUTLINE_L2 结果
cache.save_json('outline_analysis_L2/outlines_L2', outlines_L2)

#### <font color="red"> 3. 将Outline_L2 应用到tender_file_extraction_L1中</font>



In [16]:
# 应用outline_L1分析结果，数据准备： outline_L1, index_path_map_l1, tender_file_extraction 
import json
outlines_L2 = cache.load_json('outline_analysis_L2/outlines_L2')
meta_l2 = cache.load_json('outline_analysis_L2/meta_l2')
tender_file_extraction_L1 = cache.load_json('tender_file_extraction_L1')
index_path_map_L2 = meta_l2['index_path_map']

In [ ]:
# 解析OUTLINE_L2 每个JSON字符串并合并
outlines_L2_dict = []
for outline in outlines_L2:
    parsed = json.loads(outline)
    outlines_L2_dict.extend(parsed)

# 将所有的'level'值加1
for item in outlines_L2_dict:
    if 'level' in item:
        item['level'] += 1

from pprint import pprint
pprint(outlines_L2_dict)

In [ ]:
# 将outline_L1 应用到tender_file_extraction中, 使用TiptapUtils.update_titles_from_list
from apps.projects.tiptap.helpers import TiptapUtils
updated_tiptap_content = TiptapUtils.update_titles_from_list(tender_file_extraction_L1, outlines_L2_dict, index_path_map_L2)
# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(updated_tiptap_content)
from pprint import pprint
#pprint(headings)
print(TiptapUtils.print_headings(updated_tiptap_content))

# 存储tender_file_extraction_L2 结果
cache.save_json('tender_file_extraction_L2', updated_tiptap_content)


In [ ]:
from apps.projects.tiptap.client import TiptapClient
md_doc = TiptapClient().json_to_markdown(updated_tiptap_content)
html_doc = TiptapClient().json_to_html(updated_tiptap_content)
pprint(md_doc)
from IPython.display import display, HTML, Markdown
display(Markdown(md_doc["data"]))

In [ ]:
pprint(html_doc)
from IPython.display import display, HTML, Markdown
display(HTML(html_doc["data"]))

In [ ]:
html_doc_back = TiptapClient().markdown_to_html(md_doc["data"])
display(HTML(html_doc_back["data"]))

## PART-3. 构建增强型目录结构 （添加图表目录）

In [23]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()
# 添加cache 
# 以下要求NotebookCacheManager.py 和 cache目录都在 当前测试文件的相同目录下（在notebook目录下）
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e2")

from pprint import pprint

# cache使用示例： 
# cache.save('存储文件名', 参数)，  a_loaded = cache.load('存储文件名')， 存在pkl文件下，点开为乱码
# cache.save_json, cache.load_json, 存在json文件下，点开可读
# cache.save_md, cache.load_md，存在md文件下，点开可读
# cache.save_json('outline1/test1') 可构建二级目录

In [ ]:
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_L2 = cache.load_json('tender_file_extraction_L2')
print(TiptapUtils.print_headings(tender_file_extraction_L2))

#### <font color="red"> 1. 准备图表分析数据</font>

In [3]:
# extract tables 已经用在了outline_analysis_tb.py 的 prepare_context 中 
# from apps.projects.tiptap.helpers import TiptapUtils
# tables, index_path_map = TiptapUtils.extract_tables_to_markdown(tender_file_extraction_L2)
# pprint(tables[0]["markdown"])

In [25]:
# 集成时，用以下代替，模型传参 会先存储到数据库。
from apps.projects.services.tasks_preparation.outline_analysis_tb import OutlineAnalysisTb
# #获取LLM分析相关数据
data_cooker_tb = OutlineAnalysisTb(tender_file_extraction_L2)
model_params_tb, tasks_tb, meta_tb = data_cooker_tb.output_params()
# # 模拟的prompt, 
simulated_prompts_tb, formatted_messages_tb = data_cooker_tb.simulate_prompt()
cache.save_json('outline_analysis_tb/model_params_tb', model_params_tb)
cache.save_json('outline_analysis_tb/tasks_tb', tasks_tb)
cache.save_json('outline_analysis_tb/meta_tb', meta_tb)
cache.save_json('outline_analysis_tb/formatted_messages_tb', formatted_messages_tb)

In [ ]:
pprint(tasks_tb[1])

#### <font color="red"> 2. 执行图表分析</font>

In [ ]:
# 构建LLM分析器 并 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
analyzer = LLMService(model_params_tb)
outlines_tb = await analyzer.process_with_limit(tasks_tb, limit=5)
pprint(outlines_tb)

In [ ]:
# 将图表分析输出的结果 进行标记清洗 和 合并， 输出JSON dict 格式
from apps.projects.services.task_service import process_concurrent_JSON_outputs
cleaned_outlines_tb = process_concurrent_JSON_outputs(outlines_tb)
pprint(cleaned_outlines_tb)
cache.save_json('outline_analysis_tb/outlines_tb', cleaned_outlines_tb)

#### <font color="red"> 2. 应用图表目录，构建增强型目录结构</font>

In [30]:
# 应用outline_L1分析结果，数据准备： outline_L1, index_path_map_l1, tender_file_extraction 
import json
captions = cache.load_json('outline_analysis_tb/outlines_tb')
meta_tb = cache.load_json('outline_analysis_tb/meta_tb')
tender_file_extraction_L2 = cache.load_json('tender_file_extraction_L2')
index_path_map_tb = meta_tb['index_path_map']

In [ ]:
# 给节点添加caption note
from apps.projects.tiptap.helpers import TiptapUtils
updated_tiptap_content = TiptapUtils.add_captions_to_nodes(tender_file_extraction_L2, captions, index_path_map_tb)
pprint(updated_tiptap_content)
cache.save_json('tender_file_extraction_L3', updated_tiptap_content)

In [ ]:
#打印 增加型文档大纲框架 
from apps.projects.tiptap.helpers import TiptapUtils
result=TiptapUtils.print_enhanced_toc(updated_tiptap_content)
print(result)

#### <font color="red"> 3. 添加前言标题</font>

In [ ]:
tender_file_extraction_L3 = cache.load_json('tender_file_extraction_L3')
pprint(tender_file_extraction_L3)

In [ ]:
from apps.projects.tiptap.helpers import TiptapUtils
updated_tiptap_content = TiptapUtils.add_introduction_headings(tender_file_extraction_L3)
pprint(updated_tiptap_content)
cache.save_json('tender_file_extraction_L4', updated_tiptap_content)

In [ ]:
from apps.projects.tiptap.helpers import TiptapUtils
result=TiptapUtils.print_enhanced_toc(updated_tiptap_content)
print(result)

## PART-4. 章节定位分析 

In [1]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()
# 添加cache 
# 以下要求NotebookCacheManager.py 和 cache目录都在 当前测试文件的相同目录下（在notebook目录下）
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e2")

from pprint import pprint

# cache使用示例： 
# cache.save('存储文件名', 参数)，  a_loaded = cache.load('存储文件名')， 存在pkl文件下，点开为乱码
# cache.save_json, cache.load_json, 存在json文件下，点开可读
# cache.save_md, cache.load_md，存在md文件下，点开可读
# cache.save_json('outline1/test1') 可构建二级目录

Settings从哪里加载？: config.settings.development
项目根目录对么？: /home/oscarwang/BidPilot_new/backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- apps.testground
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
tender_file_extraction_L4 = cache.load_json('tender_file_extraction_L4')
# pprint(tender_file_extraction_L4)
from apps.projects.tiptap.helpers import TiptapUtils
result=TiptapUtils.print_enhanced_toc(tender_file_extraction_L4)
print(result)

[H1] 第一部分 项目基本情况 (path:[6])
  [H2] 前言 (path:[7])
  [H2] 项目概述 (path:[8])
  [H2] 招采内容 (path:[10])
  [H2] 报价范围 (path:[12])
  [H2] 服务验收 (path:[14])
  [H2] 服务人员 (path:[16])
  [H2] 服务范围的调整 (path:[18])
  [H2] 转让和分包 (path:[20])
  [H2] 知识产权 (path:[22])
  [H2] 知识产权 (path:[23])
[H1] 第二部分 报价须知 (path:[24])
  [H2] 前言 (path:[25])
  [H2] 报价须知前附表 (path:[26])
    [表] 招标采购相关事项及说明 (path:[27])
  [H2] 报价人资格 (path:[28])
  [H2] 报价费用 (path:[36])
  [H2] 现场踏勘 (path:[38])
  [H2] 招采文件说明 (path:[40])
  [H2] 招采文件的澄清与修改 (path:[42])
  [H2] 报价语言 (path:[44])
  [H2] 报价文件的质量标准 (path:[46])
  [H2] 报价文件的组成及要求 (path:[48])
  [H2] 报价文件的组成及要求 (path:[50])
    [H3] 技术文件包含以下内容 (path:[52])
  [H2] 报价文件的递交 (path:[54])
  [H2] 报价货币 (path:[56])
  [H2] 报价有效期 (path:[58])
  [H2] 报价保证金 (path:[60])
  [H2] 报价保证金的退还 (path:[61])
  [H2] 评选 (path:[64])
  [H2] 招采人拒绝报价的权力 (path:[66])
  [H2] 招采人终止程序的权力 (path:[68])
  [H2] 中选通知及合同授予 (path:[70])
  [H2] 其他报价须知 (path:[72])
[H1] 第三部分 需求说明书 (path:[75])
  [表] 全国零食包项目服务范围及要求说明表格 (path:[77])
[H1] 第四部分 附件 (path:

In [3]:
# 集成时，用以下代替，模型传参 会先存储到数据库。
from apps.projects.services.tasks_preparation.bid_writing_positioning import BidWritingPositioning
# #获取LLM分析相关数据
data_cooker_bid = BidWritingPositioning(tender_file_extraction_L4)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()
cache.save_json('bid_writing_positioning/model_params_bid', model_params_bid)
cache.save_json('bid_writing_positioning/tasks_bid', task_bid)
cache.save_json('bid_writing_positioning/meta_bid', meta_bid)
cache.save_json('bid_writing_positioning/formatted_messages_bid', formatted_messages_bid)

In [ ]:
pprint(task_bid)
pprint(meta_bid)

In [5]:
pprint(formatted_messages_bid)

[{'content': '你是一个专业的招标文档分析助手，帮助用户分析文档的结构和内容。', 'role': 'system'},
 {'content': '\n'
             '你将执行以下任务：\n'
             '\n'
             '【任务目标】\n'
             '\n'
             '以下提供了招标文件的完整目录（材料A）。。\n'
             '\n'
             '请根据语义为我确定，是否存在一个章节详细介绍了投标文件的组成结构？\n'
             '\n'
             '如果没有，请告诉我哪些章节涵盖了投标文件的组成结构。 \n'
             '\n'
             '请精准地指出章节位置，列出章节的标题。 \n'
             '\n'
             '\n'
             '\n'
             '【输出格式】\n'
             '\n'
             '        \n'
             '- 只输出符合JSON格式的数据，不要添加解释、注释或 Markdown 标记。\n'
             '- 示例：\n'
             '[\n'
             '    {"path": [int], "level": int, "title": str, "原因": str}, \n'
             '    {"path": [int], "level": int, "title": str, "原因": str}\n'
             ']\n'
             '- 一个标题一条数据。 \n'
             '\n'
             '\n'
             '\n'
             '以下是你将使用的内容：\n'
             '\n'
             '【材料A：主要上下文】\n'
             '[H1] 第一部分 项目基本情况 (path:[6])\n'
 

In [6]:
# 构建LLM分析器 并 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
analyzer = LLMService(model_params_bid)
outlines_bid = await analyzer.process(task_bid)
pprint(outlines_bid)

HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
[
    {"path": [48], "level": 2, "title": "报价文件的组成及要求", "原因": "详细介绍了投标文件的组成结构"},
    {"path": [50], "level": 2, "title": "报价文件的组成及要求", "原因": "可能包含投标文件的组成结构"}
]('[\n'
 '    {"path": [48], "level": 2, "title": "报价文件的组成及要求", "原因": '
 '"详细介绍了投标文件的组成结构"},\n'
 '    {"path": [50], "level": 2, "title": "报价文件的组成及要求", "原因": '
 '"可能包含投标文件的组成结构"}\n'
 ']')
